# Q1. Getting the embeddings model

In [7]:
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [29]:
user_question = "I just discovered the course. Can I still join it?"
q_v = embedding_model.encode(user_question)

In [30]:
print(f"The first value of the resulting vector is: {q_v[0]}") #0.07

The first value of the resulting vector is: 0.07822262495756149


# Q2. Creating the embeddings

In [11]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [12]:
documents[:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'}]

In [14]:
# We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
    
    
len(filtered_documents) #confirmed only 375 documents

375

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

Create a list embeddings
- Iterate over each document
- qa_text = f'{question} {text}'
- compute the embedding for qa_text, append to embeddings
- At the end, let X = np.array(embeddings) (import numpy as np)

What's the shape of X? (X.shape). Include the parantheses.

In [31]:
embeddings = []

for doc in tqdm(filtered_documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))

100%|██████████| 375/375 [00:07<00:00, 53.09it/s]


In [32]:
import numpy as np

X = np.array(embeddings)

print(f"Shape of the resulting array: {X.shape}")

Shape of the resulting array: (375, 768)


# Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix `X` by the vector `v`:

In [36]:
scores = X.dot(q_v)

In [37]:
scores.max() #0.65

0.65065736

# Q4. Hit-rate for our search engine

In [40]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [41]:
search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(q_v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [39]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [42]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [43]:
def evaluate(ground_truth, search_function, embedding_model):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        v = embedding_model.encode(q['question'])
        results = search_function(v)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5.

In [44]:
evaluate(ground_truth, search_engine.search, embedding_model)

100%|██████████| 1830/1830 [00:34<00:00, 53.42it/s]


{'hit_rate': 0.9579234972677596}

# Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

Create the index with the same settings as in the module (but change the dimensions)
- Index the embeddings (note: you've already computed them)
- After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [45]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

  0%|          | 0/3 [36:30<?, ?it/s]


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [46]:
for i,doc in tqdm(enumerate(filtered_documents)):
    doc['question_text_vector'] = embeddings[i]
    es_client.index(index=index_name, document=doc)

375it [00:03, 96.50it/s] 


In [47]:
es_client.count(index=index_name)

ObjectApiResponse({'count': 375, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [55]:
search_query = {
    "knn": {
        "field": 'question_text_vector',
        "query_vector": q_v,
        "k": 5,
        "num_candidates": 10000,
    },
    "_source": ["text", "section", "question", "course", "id"]
}

In [56]:
es_results = es_client.search(
        index=index_name,
        body=search_query
    )

In [62]:
es_results['hits']['hits'][0]

{'_index': 'course-questions',
 '_id': 'FbGZxZABMljuSJn22f1J',
 '_score': 0.8253289,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

In [48]:
def elastic_search_knn_small(query_vector):
    search_query = {
        "knn": {
            "field": 'question_text_vector',
            "query_vector": query_vector,
            "k": 5,
            "num_candidates": 10000,
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [54]:
evaluate(ground_truth, elastic_search_knn_small, embedding_model)

100%|██████████| 1830/1830 [00:52<00:00, 34.79it/s]


{'hit_rate': 0.9398907103825137}